## NOTES

#### RESOURCES

In [ ]:
https://brennan.io/2016/03/02/logging-in-with-requests/
https://kazuar.github.io/scraping-tutorial/

In [2]:
import json
import pandas as pd
import csv
import datetime
import collections
import re

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

pprint('all done')

'all done'


#### COOKIES & LOGIN

In [ ]:
NOT NECCESSARY B/C WE ARE IMPORTING PREVIOUSLY SCRAPED RAW HTML
However, see Prison Talk Scraper Login Notebook for process.

In [3]:
html_page = open('examples/members_list.html', 'r').read()
members_list_page = str(html_page)
no_literals = members_list_page.replace('\n', '').replace('\t', '').replace('\r', '')
members_list = BeautifulSoup(no_literals, 'html.parser')

#### REMOVE ALPHA LINKS, SEPPERATE HEADERS AND MEMBERS

In [218]:
# remove rows[0] since it is the alpha links
rows = members_list.select('table.tborder > tr[align=center]')[1:]

# get headers (e.g. 'user_name', u'join_date', ..., u'birthday')
headers = get_headers(rows[0])

# remove header row
members = rows[1:]

print 'MEMBERS IN LIST:', len(members) # MEMBERS IN LIST: 100
print ''
print 'HEADERS:', headers # HEADERS: [u'user_name', u'join_date', u'posts', u'last_visit', u'birthday']

MEMBERS IN LIST: 100

HEADERS: [u'user_name', u'join_date', u'posts', u'last_visit', u'role', u'profile_picture', u'birthday']


#### EXTRACT HEADERS FROM FIRST ROW AND CONVERT TO UNDERSCORE CASE

In [325]:
def get_headers(header_els):
    
    # grab column headers (e.g. 'User Name', 'Home Page', ..., Birthday) from a bunch of <td><a>...blah...</a><td>
    col_headers = [col_header.text.strip() for col_header in header_els.select('td')] 
    
    # We only need...[u'User Name', u'Join Date', u'Posts', u'Last Visit', u'Birthday', 'Profile Picture']
    # We have to add 'role' e.g. moderator, admin, site mod, etc.
    col_headers = [u'member_id', col_headers[0]] + col_headers[3:6] + [u'role'] + col_headers[-2:]
    
    # Put them in the form [u'user_name', u'join_date', u'posts', u'last_visit', u'birthday']
    headers = [col_header.replace(' ', '_').lower() for col_header in col_headers]
    
    return headers

# [u'user_name', u'join_date', u'posts', u'last_visit', 'role', u'birthday', 'profile_picture']
# headers = get_headers(rows[0])
# headers

#### CLEAN ROWS OF UNNECESSARY COLUMNS

In [326]:
# 0: MEMBER ID, ROLE
# 1: HOME PAGE (no)
# 2: FIND POSTS (no)
# 3: JOIN DATE
# 4: POSTS
# 5: LAST VISIT
# 6: AVATAR (no) <--- MAYBE CHANGE LATER
# 7: PROFILE PIC
# 8: BIRTHDAY

In [327]:
def clean_rows(rows):
    
    cleaned_rows = []
    
    for row in rows:
        
        cols = list(row.find_all('td'))

        cleaned_row = [ cols[0] ] + cols[3:6] + cols[-2:] 

        cleaned_rows.append(cleaned_row)
        
    return cleaned_rows

cleaned = clean_rows(members)
len(cleaned)

#### PARSE HTML ROWS FOR WRITING TO CSV

In [322]:
def parse_rows(rows):
    
    user_data = []
    
    for row in rows:
        
        row_data = {}
        
        row_data['member_id'] = get_member_id(row[0])        
        row_data['user_name'] = get_username(row[0])
        row_data['role'] = get_user_role(row[0])
        row_data['join_date'] = get_join_date(row[1])
        row_data['posts'] = get_num_posts(row[2])
        row_data['last_visit'] = get_last_visit(row[3])
        row_data['profile_pic'] = get_profile_pic(row[4])
        row_data['birthday'] = get_birthdate(row[5])

        user_data.append(row_data)
            
    return user_data

# #[{'birthday': u'August 29, 1962','join_date': u'11-20-2003','last_visit': u'09-01-2014',
# #  'member_id': u'11268','posts': u'38,207','profile_pic': 'N','role': u'red','user_name': u'Patty'},
# # {'birthday': u'July 7, 1965','join_date': u'06-26-2003','last_visit': u'03-18-2017',
# #  'member_id': u'14451','posts': u'36,365','profile_pic': 'Y','role': u'black','user_name': u'LeaAnn'}]
# parse_rows(cleaned[:2])

#### USER NAME

In [298]:
def get_username(el):

#     <i>    Patty       </i>
    username = el.a.text
    
    return username.strip()
    
# data = members[7].select('td') 
get_username(cleaned[0][0]) # u'Patty'

u'Patty'

#### USER ROLE (INFERRED FROM COLOR)

In [316]:
# Administrator: red
# PTO Super Moderator: blue
# PTO Site Moderator: purple
# PTO Moderator: darkgreen
# Moderator On Leave: magenta
# PTO Card Swap Host: MediumTurquoise
# User: black

def get_user_role(el):
    
    try:
        # <font color="red"><i>'   Patty    '</font>
        user_role = el.a.font['color']
    
    # if user is simply 'registered' then it won't have <font> element
    except TypeError as err:
        
        user_role = u'black'
        
    return user_role

get_user_role(cleaned[0][0]), get_user_role(cleaned[1][0]) # (u'red', 'black')

(u'red', u'black')

#### MEMEBER ID

In [313]:
#there are other places to get this but it's nice to use each column

def get_member_id(el):
    
    # 'u'member.php?u=91487''
    href = el.find_all('a')[0]['href']
    mid = href.split('=')[-1]

    return mid

get_member_id(cleaned[6][0]) # '91487'

u'106823'

#### JOIN DATE

In [249]:
def get_join_date(el):
    
    # u'    03-18-2007   '
    join_date = el.text.strip()
    
    # Convert 'Today' into a M-D-Y
    if join_date == 'Today':
        
        today = today = datetime.datetime.now()
        
        # everything else is in unicode so...
        join_date = unicode(today.strftime("%m-%d-%Y"))
        
    # Convert 'Yesterday' into a M-D-Y
    elif join_date =='Yesterday':
        
            yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
            join_date = unicode(yesterday.strftime("%m-%d-%Y"))
    
    return join_date

# if not Today, u'11-20-2003' else today's date, or yesterday's
get_join_date(cleaned[0][1]) 

u'11-20-2003'

#### NUM POSTS

In [250]:
def get_num_posts(el):
    
    num_posts = join_date = el.text.replace('\n', '').strip()
    
    return num_posts

get_num_posts(cleaned[0][2]) # u'38,207'

u'38,207'

#### LAST VISIT

In [299]:
def get_last_visit(el):

    # <td class="alt2">    09-01-2014     </td>
    last_visit = el.text.strip()

    if last_visit == 'Today':

        today = today = datetime.datetime.now()

        # everything else is in unicode so...
        last_visit = unicode(today.strftime("%m-%d-%Y"))

        # Convert 'Yesterday' into a M-D-Y
    elif last_visit =='Yesterday':

        yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
        last_visit = unicode(yesterday.strftime("%m-%d-%Y"))

    return last_visit


reg = cleaned[1][3]
today = BeautifulSoup('<td class="alt2">    Today     </td>', 'html.parser')
yesterday = BeautifulSoup('<td class="alt2">    Yesterday     </td>', 'html.parser')
na = BeautifulSoup('<td class="alt2">         </td>', 'html.parser')

examples = [reg, today, yesterday, na]

# [u'09-01-2014', u'02-16-2018', u'02-15-2018', u'']
[get_last_visit(example) for example in examples]

[u'03-18-2017', u'02-16-2018', u'02-15-2018', u'']

#### PROFILE PICTURE

In [307]:
def get_profile_pic(el):
    
    # Just want to know if they have one, don't care what it is.
    if el.img:
        profile_pic = 'Y'
    else:
        profile_pic = 'N'
        
    return profile_pic
    
# n = cleaned[0][4] # ID: 11268 (Patty)
# y = cleaned[1][4] # ID: 14451 (LeaAnn)

# examples = [n, y]
# [get_profile_pic(example) for example in examples] # ['N', 'Y']

['N', 'Y']

#### BIRTHDAY

In [314]:
def get_birthdate(el):
    
    birthdate = el.text.replace('\n', '').strip()
    
    return birthdate

# Assuming Bday is not today
does = cleaned[0][5] # ID: 11268 (Patty) 'August 29, 1962'
doesnt = cleaned[6][5] # ID: 106823 (sidewalker) u''
part = cleaned[7][5] # ID: 1658 (Lysbeth) u'March 15'

birthday_types = [does, doesnt, part]

# [u'August 29, 1962', u'', u'March 15']
[get_birthdate(birthday_type) for birthday_type in birthday_types]

[u'August 29, 1962', u'', u'March 15']

#### NO LONGER USING

In [386]:
# NOT USING: NEEDS REWRITE TO ADHERE TO NEW STANDARDS ABOVE
def get_avatar(el):
    
    if el.img:
        avatar = 'Y'
    else:
        avatar = 'N'
        
    return avatar

# Just want to know if they use one
# y = members[6].select('td')[6]
# n = members[7].select('td')[6]

# examples = [y, n]
# [get_avatar(example) for example in examples] # ['Y', 'N']

['Y', 'N']

In [1]:
# NOT USING: NEEDS REWRITE TO ADHERE TO NEW STANDARDS ABOVE
def get_homepage(el):
    
    if(el.a):
        homepage = el.a['href']
    else:
        return '' # will be filled later in pandas
    
    return homepage

# get_hompage(data[1]) # '' or 'http://www.myspace.com/leaann65'